In [4]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from scipy.integrate import odeint
import time
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'


In [7]:
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Input, Dense, Concatenate, Add, Multiply,Average, Subtract
from tensorflow.keras.models import Model
import traffic_data_generator
import traffic_util

In [8]:
import random as rn

rand_seed=116
rn.seed(rand_seed)
np.random.seed(rand_seed)
tf.random.set_seed(rand_seed)


In [9]:
example_name="traffic"
data_dir=traffic_data_generator.set_up(example_name)

In [10]:

delta_gen=0.05
n_traj=25000
n_samples=100000


In [11]:
#===========================
#     Output directories 
#===========================
experiment_name="_no_share"+ \
     "_"+str(n_samples)+"_Delta_"+str(delta_gen)


model_dir=example_name+"/"+experiment_name
plot_dir=model_dir+"/Plots"
checkpoint_dir=model_dir+"/check_points"

if not os.path.exists(model_dir):
    os.mkdir(model_dir)
if not os.path.exists(plot_dir):
    os.mkdir(plot_dir)
if not os.path.exists(checkpoint_dir):
    os.mkdir(checkpoint_dir)

In [12]:
combinations=np.array([[64,.25],[40,.25],[24,.25],[16,.25],[8,.25],[32,.5],[20,.5],[12,.5],[8,.5],[4,.5],
                       [16,1],[10,1],[6,1],[4,1],[2,1],[8,2],[5,2],[3,2],[2,2]])
n_new_combo=len(combinations)
n_bins=10
bin_size=13
ith_run=12
domain=np.load(data_dir+'/parameters AW'+str(ith_run)+ 'd_gen='+str(delta_gen) + 'n_samples='+str(n_samples)+'.npy')
traj_len=int(np.random.uniform(domain[9,0],domain[9,1])) # total time 
n_traj_per_run=50
n_runs=int(np.ceil(n_traj/n_traj_per_run))
traj_dir=data_dir+ '/parameters AW'+str(ith_run)+ 'd_gen='+str(delta_gen) + 'n_samples='+str(n_samples)
assert os.path.exists(traj_dir)
#parameter check
domain_key=('n_cars','T_r','C','rho_max','v_max','vel_init_mean','vel_variance as fraction of mean','separation mean',
           'separation variance as a fraction of mean','traj_length','indicator')
for i in range(len(domain)):
    print(domain_key[i],domain[i])
print('ith_run',ith_run)

n_cars [300. 300.]
T_r [5. 5.]
C [1. 1.]
rho_max [0.8 0.8]
v_max [1.3 1.3]
vel_init_mean [1.15 1.15]
vel_variance as fraction of mean [0.5 0.5]
separation mean [0.4 0.4]
separation variance as a fraction of mean [0.06547038 0.06547038]
traj_length [20. 20.]
indicator [0. 0.]
ith_run 8


In [13]:
def data_save_path(i_combo,Delta_t,len_hist_MZ,ith_run,var,prefix):
    path= data_dir+'/'+prefix+var+'_delta_t='+str(Delta_t)+ 'MZ_hist='+str(len_hist_MZ) +'ith run'+str(ith_run)+'_'+ str(i_combo)+'.npy'
    return path


In [14]:
for prefix in ("abs_weight_smoothed_",""):
    for i_combo in range(len(combinations)):
        len_hist_MZ=int(combinations[i_combo,0])
        Delta_t=combinations[i_combo,1]
        delta_ratio=int(round(Delta_t/delta_gen))
        x=np.load(data_save_path(i_combo,Delta_t,len_hist_MZ,ith_run,'x',prefix))
        y=np.load(data_save_path(i_combo,Delta_t,len_hist_MZ,ith_run,'y',prefix))
        new_len_hist_MZ=int(len_hist_MZ/2)
        new_Delta_t=Delta_t*2
        new_delta_ratio=int(round(new_Delta_t/delta_gen))
        if new_len_hist_MZ in combinations[combinations[:,1]==new_Delta_t,0]:
            new_i_combo=np.intersect1d(np.where(combinations[:,1]==new_Delta_t),np.where(combinations[:,0]==new_len_hist_MZ))[0]
            if not os.path.exists(data_save_path(new_i_combo,new_Delta_t,new_len_hist_MZ,ith_run,'x',prefix)): 
                x_new=x[:,1::2,:]
                y_new=y
                np.save(data_save_path(new_i_combo,new_Delta_t,new_len_hist_MZ,ith_run,'x',prefix),x_new)
                np.save(data_save_path(new_i_combo,new_Delta_t,new_len_hist_MZ,ith_run,'y',prefix),y_new)

In [28]:
#polynomial fit trajectory build
def poly_fit_eval(x, y, deg):
    p = np.polyfit(x, y, deg)  # the polynomial coefficients after Least squares polynomial fitting.
    y_new = np.polyval(p, x)  # Evaluate a polynomial at specific values.
    return y_new


# In[] Loading the data

for i_combo in range(len(combinations)):
    len_hist_MZ=int(round(combinations[i_combo,0]))
    Delta_t=combinations[i_combo,1] 
    delta_ratio=int(round(Delta_t/delta_gen))
    x = np.load(f'{data_dir}/x_delta_t={Delta_t}MZ_hist={len_hist_MZ}ith run{ith_run}_{i_combo}.npy')
    y = np.load(f'{data_dir}/y_delta_t={Delta_t}MZ_hist={len_hist_MZ}ith run{ith_run}_{i_combo}.npy')
    n_samples = x.shape[0]
    n_bins = x.shape[2]  # 10


    # In[] Data transformation by fitting the 5th polynomial
    deg = 5
    bin_size = 13
    bin_edge = -np.arange(n_bins)*bin_size
    x_new = np.zeros_like(x)
    y_new = np.zeros_like(y)
    for i_sample in range(n_samples):
        if i_sample%5000==0:print(i_sample)
        for i_x in range(len_hist_MZ):
            x_new[i_sample, i_x, :] = poly_fit_eval(bin_edge, x[i_sample, i_x, :], deg)
        y_new[i_sample, :] = poly_fit_eval(bin_edge, y[i_sample, :], deg)


    # In[] Store the data
    np.save(f'{data_dir}/poly_deg={deg}_x_delta_t={Delta_t}MZ_hist={len_hist_MZ}ith run{ith_run}_{i_combo}.npy', x_new)
    np.save(f'{data_dir}/poly_deg={deg}_y_delta_t={Delta_t}MZ_hist={len_hist_MZ}ith run{ith_run}_{i_combo}.npy', y_new)


0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
7